### 前言
spark shuffle 演进的历史
* Spark 0.8及以前 Hash Based Shuffle
* Spark 0.8.1 为Hash Based Shuffle引入File Consolidation机制
* Spark 0.9 引入ExternalAppendOnlyMap
* Spark 1.1 引入Sort Based Shuffle，但默认仍为Hash Based Shuffle
* Spark 1.2 默认的Shuffle方式改为Sort Based Shuffle
* Spark 1.4 引入Tungsten-Sort Based Shuffle
* Spark 1.6 Tungsten-sort并入Sort Based Shuffle
* Spark 2.0 Hash Based Shuffle退出历史舞台 

```
hash shuffle曾是spark shuffle中最原始的方案, 曾出过2个版本:
(1) 第一版是每个map任务都会产生reduce任务个数个文件, shuffle write阶段共产生mapNums*reduceNums个数个文件. 这个版本中因为产生的文件个数太多影响了扩展性  
(2) 第二版是让1个core产生一个文件, 被分到一个core上的任务共用一个文件; 意思就是说, 每当需要spill的时候, 同一个core都会把数据spill到同一个文件上, 这时候共产生coreNums个数个文件.  虽然相比第一版shuffle write出的文件已经少了很多, 但中间的文件个数仍然会随着集群增大而线性增大

```


### 一. Shuffle过程总体思路
目前版本的shuffle, 都是使用排序相关的shuffle; 整体上spark shuffle分为shuffle read和shuffle write:   
#### 1.  **shuffle write阶段**   
大体上经过排序, 聚合, 归并(多个文件spill磁盘的情况), 最终, 每个task生成2种文件: 数据文件和索引文件. 
* 数据文件: 相同分区的数据在文件内是连续的, 
* 索引文件: 记录了每个分区的开始位置和结束位置  
  
#### 2.  **shuffle read阶段**   
* 首先, 通过网络从各个writer节点获取给定分区的数据, 即数据文件中某一连续区域的数据, 然后再将这些不同节点下的分区文件进行排序, 合并; 最终形成分区下的结果
* shuffle write阶段需要落盘(spill), shuffle read阶段同样需要落盘(将不同机器上fetch到的文件merge到一起)


## 二. 三种shuffle writer的过程 
#### 前言
* 以下三个类， 全都是ShuffleWriter的子类， 这些子类的write()方法用于执行shuffle write。
* ShuffleMapTask的runTask()表示任务的执行, 方法内部会根据情况选择一种ShuffleWriter执行shuffle write操作; shuffle write操作的本质是当算子计算出一个内存集合后, 将这个内存集合写到一个提供将数据"分区,排序,合并,聚合"的数据结构中. 这个数据结构在SortShuffleWriter实现中, 表现为`PartitionedAppendOnlyMap`和`PartitionedPairBuffer`     

```scala
/** ShuffleWriter顶层接口声明 */
private[spark] abstract class ShuffleWriter[K, V] {
  /** Write a sequence of records to this task's output */
  @throws[IOException]
  def write(records: Iterator[Product2[K, V]]): Unit

  /** Close this writer, passing along whether the map completed */
  def stop(success: Boolean): Option[MapStatus]
}

```

### 2.1 BypassMergeSortShuffleWriter   
1. BypassMergeSortShuffleWriter**实现过程**   
    因为, BypassMergeSortShuffleWriter大体实现过程是: 首先为每个分区创建一个分区文件, 将对应的数据写入到各自的分区文件中; 最终合并所有的分区文件形成一个大文件, 且伴随产生一个索引文件记录分区的开始位置和结束位置. 因为这种shuffle不需要进行排序和combine等操作, 因此不会十分消耗内存.
  
       
2. BypassMergeSortShuffleWriter的**触发条件**有2个: 
    1. map task的数量不多: 小于`spark.shuffle.sort.bypassMergeThreshold`值, 默认为200
    2. 非map端聚合的算子: 比如`reduceByKey`就不能使用bypasssertshuffle.
    
    
### 2.2 SortShuffleWriter
#### 1. sort based shuffle流程:     
SortShuffleWriter是日常使用最频繁的shuffle过程; SortShuffleWriter主要使用`ExternalSorter`对数据进行排序, 合并, 聚合(combine). 最后产生数据文件和索引文件
* (1)**首先**, 根据算子是否需要map端进行combine, 数据会被写入到不同的数据结构: 
    1. 需要进行combine的算子, 数据写入到`PartitionedAppendOnlyMap`这个map中
    2. 不需要进行combine的算子, 数据写入到`PartitionedPairBuffer`这个数组中
    
    
* (2)**然后**, 每个一段时间, 向MemoryManager申请内存
    * 如果可以一直申请到内存, 则`PartitionedAppendOnlyMap`和`PartitionedPairBuffer`中的数据会一直存在于内存中, 可见这两个数据结构很消耗内存.
    * 如果发现申请不到了, 则会把内存中的数据spill到磁盘文件; 在spill到文件之前, 需要先把`PartitionedAppendOnlyMap`和`PartitionedPairBuffer`中的数据和已经spill到磁盘的数据进行合并(使用归并排序合并). `PartitionedAppendOnlyMap`和`PartitionedPairBuffer`内存中的排序算法使用[TimSort]()算法    


* (3)**最后**, 写入文件一般是先把数据写入一个buffer, 再把buffer中的数据写入到文件. 此过程受2个参数控制  
    * [spark.shuffle.file.buffer](#param1)
    * spark.shuffle.spill.batchSize 


```
综上所述, 普通的sort shuffle, 
  (1) 在shuffle write阶段共产生**文件个数: map任务的个数**(不算索引文件)
  (2) 并发持有并且进行写入的文件数最多为：coreNums
```  

#### 2. 如何判断当前内存中是否还有空间, 让数据继续放入`PartitionedAppendOnlyMap`和`PartitionedPairBuffer`中?  
&nbsp;&nbsp;&nbsp;&nbsp;这个问题就是上述流程中, 第二点, MemoryManager怎么判断是否仍有内存空间留给内存中的shuffle write数据, 是否需要spill `PartitionedAppendOnlyMap`和`PartitionedPairBuffer`的数据到磁盘? 这个问题的主要难处在于, spark内存中的数据都是有用数据, 往往无法通过GC自主控制内存, 所以如果spill时机检测的不及时, 即使产生GC可能仍会导致OOM问题. 但是如果每放入`PartitionedAppendOnlyMap`和`PartitionedPairBuffer`
中一条数据就检测内存占用情况, 会导致效率极其低下. Spark如何实现呢?   
* 首先, 我们从write()方法看起, 这是执行shuffle write的起点 

```scala
// ShuffleWriter.write()
override def write(records: Iterator[Product2[K, V]]): Unit = {
    .... 
    // 这个sorter被前面代码定义成了ExternalSorter
    sorter.insertAll(records)
   ....
}
```
* 然后, 进入ExternalSorter, 查看其insertAll(), 关键在于期内部的`maybeSpillCollection()`
```scala
def insertAll(records: Iterator[Product2[K, V]]): Unit = {
    // 查看是否是需要map端聚合 (算子和map任务个数)
    val shouldCombine = aggregator.isDefined

    if (shouldCombine) {   // 1. 如果需要聚合, recorders将会被插入PartitionedAppendOnlyMap中
      // Combine values in-memory first using our AppendOnlyMap
      ....
      while (records.hasNext) {
        addElementsRead()  // 1.1 更新状态, 已经向数据结构中插入了几条recorder
        kv = records.next()
        map.changeValue((getPartition(kv._1), kv._1), update)  // 1.2 执行向集合中插入recorder的动作 
        maybeSpillCollection(usingMap = true)  // 1.3 查看是否需要spill
      }
    } else {
      // Stick values into our buffer
      while (records.hasNext) {  /**2. 如果不需要聚合, 数据将会被插入到PartitionedPairBuffer中*/
        addElementsRead()
        val kv = records.next()
        buffer.insert(getPartition(kv._1), kv._1, kv._2.asInstanceOf[C])
        maybeSpillCollection(usingMap = false)
      }
    }
  }
```
* 最后, 进入ExternalSorter的方法, 查看其内部检查是否需要spill的逻辑
    1. 首先需要得到当前map/buffer的估计内存占用大小: estimatedSize = buffer.estimateSize()
    2. 其次还要看本次是否需要检查spill: maybeSpill(buffer, estimatedSize)
```scala
/**
   * Spill the current in-memory collection to disk if needed.
   *
   * @param usingMap whether we're using a map or buffer as our current in-memory collection
   */
  private def maybeSpillCollection(usingMap: Boolean): Unit = {
    var estimatedSize = 0L
    if (usingMap) {
      estimatedSize = map.estimateSize()  // map指PartitionedAppendOnlyMap
      if (maybeSpill(map, estimatedSize)) { //
        map = new PartitionedAppendOnlyMap[K, C]
      }
    } else {
      estimatedSize = buffer.estimateSize()  // buffer指PartitionedPairBuffer
      /**
      每次底层的hashmap需要扩大2倍, 进行rehash时, 或每隔插入n隔record后, 计算每更新一条recorder占用的内存大小(bytesPerUpdate)
      (1) n是指数增加的, n每次更新成n*1.1, n的初始值是1. 使用n的时候会取n的上界(Math.ceil)
      (2) bytesPerUpdate = (上次抽样时集合的大小-上上次抽样时集合的大小) / (者两次抽样插入recorder数量之差)
         bytesPerUpdate的估算值只和前两次抽样有关. 
      (3) 计算一个集合的大小, 在SizeEstimator.estimate()中, 主要是通过遍历集合单个元素后累加得来的. 
         spark对单个对象内存占用的计算, 不仅会包含对象本身, 也包含对象应用的对象, 其引用的引用对象等等.
      (4) 最终, estimatedSize = bytesPerUpdate * (本次和上次更新的recorder差) + 上次抽样时的集合大小
      */
      if (maybeSpill(buffer, estimatedSize)) {
        /** 
        maybeSpill方法, 每隔32条recorder,检查一次
         (1)如果发现当前集合的estimatedSize超过了myMemoryThreshold阈值. 则:
            先更新myMemoryThreshold的大小, myMemoryThreshold的初始大小为5M, 每次扩大为当前estimatedSize的2倍, 如果myMemoryThreshold已经达到了(shuffle内存*安全系数), 但estimatedSize仍然要比它大, 则开始spill到磁盘. 
            shuffle内存: spark.shuffle.memoryFraction=0.2
            安全系数: spark.shuffle.safetyFraction=0.8
        */
        buffer = new PartitionedPairBuffer[K, C]
      }
    }

    if (estimatedSize > _peakMemoryUsedBytes) {
      _peakMemoryUsedBytes = estimatedSize
    }
  }
```

#### 3. 结论
我们说shuffle是可能会产生OOM的原因有2个:
1. 一个是集合大小的计算可能不准确. 集合的实际大小可能比估算大小大. 这个不准确来自2方面: 
     1. 当前集合被抽样到时, 其内部单个对象的大小计算有误差
     2. 抽样是通过每插入${1.1}^n$条recorder后, 计算的recorder平均大小
2. 第二个是检查不及时, 每隔32次insert后才去检查当前集合的大小, 因此可能还来不及检查就已经OOM了

### 2.3 UnsafeShuffleWriter
`UnsafeShuffleWriter`是`SortShuffleWriter`的优化版本,Tungsten-sort优化点主要在三个方面:
1. 直接在serialized binary data上sort而不是java objects，减少了memory的开销和GC的overhead。
2. 提供cache-efficient sorter，使用一个8bytes的指针，把排序转化成了一个指针数组的排序。
3. spill的merge过程也无需反序列化即可完成

Spark 默认开启的是Sort Based Shuffle,想要打开Tungsten-sort ,请设置
```
spark.shuffle.manager=tungsten-sort
```
对应的实现类是：
```
org.apache.spark.shuffle.unsafe.UnsafeShuffleManager
```

### 三. Shuffle Read的过程
* **过程**:  
  shuffle read主要完成数据获取和对数据的spill ,combine, merge操作. 数据获取又分为直接从本地获取和从远端拉取.如果是从远端拉取, 需要在拉去的同时spill小文件, 最后进行combine, merge文件的操作.
* **一般不会OOM**  
  shuffle read阶段由于没有估算, 检查的问题, 所以一般不会出现OOM. 都会保持好内存使用在`ExecutorHeapMemeory * 0.2 * 0.8` 之内

### 四. 结论: 什么情况下可能会导致Spark Shuffle阶段发生OOM
* **1. 并发度过高**  
 可能1个Execotor上设置的并发任务书过多 导致每个job分到的内存太少
 
* **2. 数据倾斜时, 单个文件过大, 导致拉取到内存后发生OOM**  
 在 Reduce 获取数据时，由于数据倾斜，有可能造成单个 Block 的数据非常的大，默认情况下是需要有足够的内存来保存单个 Block 的数据。因此，此时极有可能因为数据倾斜造成 OOM。 可以设置 spark.maxRemoteBlockSizeFetchToMem 参数，设置这个参数以后，超过一定的阈值，会自动将数据 Spill 到磁盘，此时便可以避免因为数据倾斜造成 OOM 的情况。在我们的生产环境中也验证了这点，在设置这个参数到合理的阈值后，生产环境任务 OOM 的情况大大减少了。
 
* **3. spill不及时**  
Spark 目前采用的是抽样统计的方式来计算 MemoryConsumer 已经使用的内存，从而造成堆内内存的实际使用量不是特别准确。从而有可能因为不能及时 Spill 而导致 OOM。